In [13]:
import tomotopy as tp
print(tp.isa)

avx2


In [14]:
# Read uspto dataset
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

usptodata = pd.read_csv('U.S. Patents.csv')
usptodataset=usptodata[["grant_id","claims_text","abstract"]]
usptodataset= usptodataset.dropna()
usptodataset = usptodataset.reset_index(drop=True)
usptodataset.head()

,grant_id,claims_text,abstract
0,USPP030977,1. A new and distinct Mango plant characterize...,"A new and distinct variety of Mango plant, her..."
1,USPP030978,1. A new and distinct apple tree substantially...,&#x2018;Honeysuckle Rose #1-6&#x2019; is a new...
2,USPP030979,1. A new and distinct variety of peach tree as...,"A new and distinct peach tree variety, <i>Prun..."
3,USPP030980,1. A new and distinct variety of raspberry pla...,This invention relates to a new and distinct v...
4,USPP030981,1. A new and distinct Strawberry plant named &...,A new and distinct cultivar of Strawberry plan...


In [15]:
#Clean and token data
stopwords = nltk.corpus.stopwords.words('english')

def lemma(word): #map pos tag
    tag = nltk.pos_tag([word])[0][1][0].upper()
    allowed_tags = {'J','N', 'V', 'R'}
    if tag in allowed_tags:
        lemma = WordNetLemmatizer().lemmatize(word)
        return lemma
    else:
        return ''
    
def  prepare_text(text):
    tokens = word_tokenize(text)
    tokens = [token  for token in tokens if len(token)> 3]
    tokens = [token for token in tokens if token not in stopwords]
    token = [lemma(token) for token in tokens]
    return tokens
    
usptodataset['Text'] = usptodataset['abstract'].astype(str)
usptodataset['Text'] = usptodataset['Text'].apply(prepare_text)


In [16]:
usptodataset.head()

,grant_id,claims_text,abstract,Text
0,USPP030977,1. A new and distinct Mango plant characterize...,"A new and distinct variety of Mango plant, her...","[distinct, variety, Mango, plant, herein, refe..."
1,USPP030978,1. A new and distinct apple tree substantially...,&#x2018;Honeysuckle Rose #1-6&#x2019; is a new...,"[x2018, Honeysuckle, Rose, x2019, variety, der..."
2,USPP030979,1. A new and distinct variety of peach tree as...,"A new and distinct peach tree variety, <i>Prun...","[distinct, peach, tree, variety, Prunus, persi..."
3,USPP030980,1. A new and distinct variety of raspberry pla...,This invention relates to a new and distinct v...,"[This, invention, relates, distinct, variety, ..."
4,USPP030981,1. A new and distinct Strawberry plant named &...,A new and distinct cultivar of Strawberry plan...,"[distinct, cultivar, Strawberry, plant, named,..."


In [40]:
#split data to have little data to run
# divide dataset to train and test
# from sklearn.model_selection import train_test_split

# df_train, df_test = train_test_split(usptodataset, test_size=0.8, random_state=25)
# df_train=df_train.reset_index(drop=True)
# df_train

In [55]:
def cleantext(df): 
    
    usptodataset['cleaned_text'] = usptodataset['abstract'].replace(r'\'|\"|\,|\.|\?|\+|\-|\/|\=|\(|\)|\n|"', '', regex=True)
    usptodataset['cleaned_text'] = usptodataset['cleaned_text'].replace("  ", " ")
    
    # convert tweets to lowercase
    usptodataset['cleaned_text'] = usptodataset['cleaned_text'].str.lower()
    
    #remove numbers
    usptodataset['cleaned_text'] =usptodataset['cleaned_text'].replace(r'\d+', '', regex = True)
        
    #remove_symbols
    usptodataset['cleaned_text']  = usptodataset['cleaned_text'].replace(r'[^a-zA-Z0-9]', " ", regex=True)
    
    #remove punctuations 
    usptodataset['cleaned_text'] = usptodataset['cleaned_text'].replace(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+',"", regex = True)
    
    #remove_URL(x):
    usptodataset['cleaned_text']  = usptodataset['cleaned_text'].replace(r'https.*$', "", regex = True)
    
    #remove stopwords and words_to_remove
    usptodataset['fully_cleaned_text'] = usptodataset['cleaned_text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stopwords]))
    
    return usptodataset

df = cleantext(usptodataset)


In [42]:
df 

,grant_id,claims_text,abstract,Text,cleaned_text,fully_cleaned_text
0,USPP030977,1. A new and distinct Mango plant characterize...,"A new and distinct variety of Mango plant, her...","[distinct, variety, Mango, plant, herein, refe...",a new and distinct variety of mango plant here...,new distinct variety mango plant herein referr...
1,USPP030978,1. A new and distinct apple tree substantially...,&#x2018;Honeysuckle Rose #1-6&#x2019; is a new...,"[x2018, Honeysuckle, Rose, x2019, variety, der...",x honeysuckle rose x is a new variety de...,x honeysuckle rose x new variety derived x sim...
2,USPP030979,1. A new and distinct variety of peach tree as...,"A new and distinct peach tree variety, <i>Prun...","[distinct, peach, tree, variety, Prunus, persi...",a new and distinct peach tree variety i prunu...,new distinct peach tree variety prunus persica...
3,USPP030980,1. A new and distinct variety of raspberry pla...,This invention relates to a new and distinct v...,"[This, invention, relates, distinct, variety, ...",this invention relates to a new and distinct v...,invention relates new distinct variety red ras...
4,USPP030981,1. A new and distinct Strawberry plant named &...,A new and distinct cultivar of Strawberry plan...,"[distinct, cultivar, Strawberry, plant, named,...",a new and distinct cultivar of strawberry plan...,new distinct cultivar strawberry plant named x...
...,...,...,...,...,...,...
7008,US10462947,"1. A component mounting machine comprising:,a ...",Provided are a first component holding tool op...,"[Provided, first, component, holding, tool, op...",provided are a first component holding tool op...,provided first component holding tool operatin...
7009,US10462948,1. A mounting deviation correction apparatus w...,In a case in which mounting deviation is occur...,"[case, mounting, deviation, occurring, compone...",in a case in which mounting deviation is occur...,case mounting deviation occurring component mo...
7010,US10462949,1. A reel holding device configured for use wi...,A reel holding device is provided. The device ...,"[reel, holding, device, provided, device, incl...",a reel holding device is provided the device i...,reel holding device provided device includes c...
7011,US10462950,1. An electronic component bonding device for ...,An electronic component bonding device include...,"[electronic, component, bonding, device, inclu...",an electronic component bonding device include...,electronic component bonding device includes m...


In [43]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

df['fully_cleaned_text'] = df['fully_cleaned_text'].apply(lemmatize_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [61]:
df['abstract']

0       A new and distinct variety of Mango plant, her...
1       &#x2018;Honeysuckle Rose #1-6&#x2019; is a new...
2       A new and distinct peach tree variety, <i>Prun...
3       This invention relates to a new and distinct v...
4       A new and distinct cultivar of Strawberry plan...
                              ...                        
7008    Provided are a first component holding tool op...
7009    In a case in which mounting deviation is occur...
7010    A reel holding device is provided. The device ...
7011    An electronic component bonding device include...
7012    A control device that determines tape referenc...
Name: abstract, Length: 7013, dtype: object

In [75]:
#save dataframe to txt file
df['fully_cleaned_text'].to_csv(r'C:\Users\admin\Topic-Modeling\U.S.Patents.txt', header=None, index=None, sep=' ', mode='a')

In [76]:
import tomotopy as tp
#class HLDAModel(tw=TermWeight.ONE, min_cf=0, min_df=0, rm_top=0, depth=2, alpha=0.1, eta=0.01, gamma=0.1, seed=None, corpus=None, transform=None)


file_name = 'U.S.Patents.txt'

mdl = tp.HLDAModel(depth=3)     
for line in open(file_name, 'r'):
    document = line.strip().split()
    mdl.add_doc(document)

print('Training model by iterating over the corpus 1 times, 1 iterations at a time')
iterations = 1
for i in range(0, 1, iterations):
    mdl.train(iterations)
    print('Iteration: #{}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    if not mdl.is_live_topic(k):
        continue
    print('child of topic #{} - Level: #{}, number of documents:#{}'.format(mdl.parent_topic(k), mdl.level(k), mdl.num_docs_of_topic(k)) )
    print('Top 10 words of global topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=5))

Training model by iterating over the corpus 1 times, 1 iterations at a time
Iteration: #0	Log-likelihood: -8.906200209575838
child of topic #-1 - Level: #0, number of documents:#35065
Top 10 words of global topic #0
[('the', 0.021609727293252945), ('a', 0.017883332446217537), ('first', 0.01587456464767456), ('second', 0.011641200631856918), ('of', 0.01076664961874485)]
child of topic #0 - Level: #1, number of documents:#31428
Top 10 words of global topic #8
[('the', 0.021762967109680176), ('a', 0.016920898109674454), ('first', 0.016507979482412338), ('second', 0.012525633908808231), ('of', 0.010183669626712799)]
child of topic #0 - Level: #1, number of documents:#248
Top 10 words of global topic #9
[('x', 0.035140473395586014), ('sub', 0.023225711658596992), ('and', 0.013734289444983006), ('plant', 0.009695385582745075), ('a', 0.009291495196521282)]
child of topic #0 - Level: #1, number of documents:#247
Top 10 words of global topic #10
[('soybean', 0.06166946515440941), ('plant', 0.03

In [77]:
mdl.summary()

<Basic Info>
| HLDAModel (current version: 0.12.3)
| 35065 docs, 2653029 words
| Total Vocabs: 47784, Used Vocabs: 47784
| Entropy of words: 7.78521
| Entropy of term-weighted words: 7.78521
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 1, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.90620
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| depth: 3 (the maximum depth level of hierarchy between 2 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-depth level, given as a single `float` in case of symmetric prior and as a list with length `depth` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| gamma: 0.1 (concentration coeficient of Dirichlet Process)
| seed: 2061585305 (random seed)
| trained in version 0.12.3
|

In [ ]:
# import tomotopy as tp
# file_name = 'U.S.Patents.txt'

# def tokenizer(x): return x.split()
# corpus = [tokenizer(s) for s in file_name]


# hLDA_tomotopy_model = tp.HLDAModel(depth=3, min_df=1, rm_top=100)
# for token_list in corpus: hLDA_tomotopy_model.add_doc(token_list)
# for i in range(0,1000,100): hLDA_tomotopy_model.train(100)
# hLDA_tomotopy_model.summary()  
  

In [ ]:
# # evaluation
# import gensim
# from gensim import corpora
# from gensim.models import CoherenceModel

# text_data = df['Text']
# dictionary = corpora.Dictionary(text_data)

In [ ]:
# hlda=mdl.vocabs
# npmi_Coherence_hLDA = CoherenceModel(model=hlda , texts=text_data, dictionary= dictionary, coherence='c_npmi') 
# print("npmi_Coherence for hdp: " + str(npmi_Coherence_hLDA.get_coherence()))

# cv_Coherence_hlda = CoherenceModel(model=hlda, texts=text_data, dictionary= dictionary, coherence='c_v') 
# print("cv_Coherence for hdp: "+ str(cv_Coherence_hlda.get_coherence()))

In [36]:
# import tomotopy as tp
# from nltk.stem.porter import PorterStemmer
# from nltk.corpus import stopwords

# stemmer = PorterStemmer()
# stopwords = set(stopwords.words('english'))  
# corpus = tp.utils.Corpus(tokenizer=tp.utils.SimpleTokenizer(stemmer=stemmer.stem), 
#     stopwords=lambda x: len(x) <= 2 or x in stopwords)
# corpus.process(open('U.S.Patents.txt', encoding='utf-8'))


2804

In [78]:
for preset in ('u_mass', 'c_uci', 'c_npmi', 'c_v'):
    coh = tp.coherence.Coherence(mdl, coherence=preset)
    average_coherence = coh.get_score()
    coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
    print('==== Coherence : {} ===='.format(preset))
    print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
    print()

==== Coherence : u_mass ====
Average: -5.376122364224454 
Per Topic: [-1.863233900458174, -1.8958693883289686, -1.8958693883289686, -1.8958693883289686, -1.8958693883289686, -1.8958693883289686, -1.8958693883289686, -1.8958693883289686, -1.909828499348637, -9.757109510499086, -3.818571418483407, -2.024440267986851, -2.8540288612953, -13.449123004142375, -1.092151670631607, -2.471776188494132, -0.9682266226779783, -1.513563364202559, -1.8912837557001123, -4.9845086258807685, -2.0368485735268234, -2.198996676126548, -0.5895519805826808, -19.54043346758385, -1.8665787976938542, -3.67223142908201, -16.45615132197839, -3.1364026477083238, -12.64187033687663, -9.683054679200033, -12.40020245126478, -9.22276701678285, -1.8922319721977336, -1.9239714313575889, -1.9171897243729716, -3.5518474870732097, -1.9054325969385937, -9.402228563579918, -7.275160629545903, -0.46354926038277017, -0.4856642351281458, -16.679069260042002, -17.82480550499744, -8.006535191199557, -1.8958693883289686, -1.895869

In [79]:
# import itertools
# for seg, cm, im in itertools.product(tp.coherence.Segmentation, tp.coherence.ConfirmMeasure, tp.coherence.IndirectMeasure):
#     coh = tp.coherence.Coherence(mdl, coherence=(tp.coherence.ProbEstimation.DOCUMENT, seg, cm, im))
#     average_coherence = coh.get_score()
#     coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]
#     print('==== Coherence : {}, {}, {} ===='.format(repr(seg), repr(cm), repr(im)))
#     print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)
#     print()